# 3) Sampling and Random Number Generator

### Basic Random Number Generator

**What is this?**
Creates a random number generator (RNG) with a seed for reproducible randomness.

**Why use a seed?**
- Makes your random experiments reproducible
- Same seed = Same sequence of "random" numbers
- Essential for debugging and scientific reproducibility

**Usage:**
```python
rng = make_rng(seed=42)  # Fixed seed
samples = rng.normal(0, 1, size=100)  # Will always give same 100 numbers
```

In [ ]:
import numpy as np

def make_rng(seed=0):
    """
    Create a random number generator with specified seed.
    
    Parameters:
    -----------
    seed : int, default=0
        The seed value for reproducibility. Same seed produces same random sequence.
    
    Returns:
    --------
    rng : numpy.random.Generator
        A random number generator object that can generate various distributions.
    """
    return np.random.default_rng(seed)

In [ ]:
# Example: Create RNG and generate samples
rng = make_rng(seed=42)
samples = rng.normal(0, 1, size=5)
print("Random samples:", samples)

### LCG (Linear Congruential Generator)

**What is this?**
A simple algorithm for generating pseudo-random numbers using a linear recurrence relation.

**Formula:** $u_{n+1} = (a \cdot u_n + c) \mod m$

**Variables:**
- $u_n$: Current state value (integer)
- $u_{n+1}$: Next state value (integer)
- $a$: Multiplier constant
- $c$: Increment constant
- $m$: Modulus (determines period and range)

**How it works:**
1. Start with a seed value $u_0$
2. Apply the formula repeatedly to generate $u_1, u_2, u_3, \ldots$
3. Normalize by dividing by $m$ to get values in $[0, 1]$

**Parameters:**
- `a = 1103515245`: Multiplier (chosen for good statistical properties)
- `c = 12345`: Increment (adds non-linearity)
- `m = 2^31`: Modulus (determines maximum period)
- These specific values are used in many standard libraries (glibc, ANSI C)

**Limitations:**
- Not cryptographically secure
- Can have patterns in high dimensions (lattice structure)
- Modern RNGs (like numpy's PCG64) are better for serious work

In [ ]:
def lcg(size, seed=1, a=1103515245, c=12345, m=2**31):
    """
    Linear Congruential Generator for pseudo-random number generation.
    
    Parameters:
    -----------
    size : int
        Number of random values to generate
    seed : int, default=1
        Initial seed value (u_0), determines the starting point of sequence
    a : int, default=1103515245
        Multiplier constant in the recurrence relation
    c : int, default=12345
        Increment constant in the recurrence relation
    m : int, default=2^31
        Modulus, determines the period and output range
    
    Returns:
    --------
    out : numpy.ndarray, shape (size,)
        Array of pseudo-random values in [0, 1]
    """
    u = seed
    out = []
    for _ in range(size):
        u = (a*u + c) % m  # Apply LCG formula
        out.append(u / m)   # Normalize to [0, 1]
    return np.array(out, float)

In [ ]:
# Example: Generate 10 pseudo-random numbers using LCG
random_numbers = lcg(size=10, seed=42)
print("LCG samples:", random_numbers)

### Inversion Sampling

**What is this?**
A method to generate random samples from any distribution by using its inverse CDF (cumulative distribution function).

**Algorithm:**
1. Generate uniform random variable $U \sim \text{Uniform}(0,1)$
2. Apply inverse CDF: $X = F^{-1}(U)$
3. Result: $X$ follows the desired distribution!

**Variables:**
- $U$: Uniform random variable on $(0, 1)$
- $F(x)$: Cumulative distribution function of target distribution
- $F^{-1}(u)$: Inverse CDF (quantile function)
- $X$: Random variable with desired distribution

**Why it works:**
- Mathematical property (Probability Integral Transform): If $U \sim \text{Uniform}(0,1)$, then $F^{-1}(U)$ has CDF $F$
- This is a fundamental theorem in probability

**When to use:**
- When you can compute or approximate the inverse CDF $F^{-1}$
- Examples: Exponential, Weibull, Cauchy distributions
- Not suitable when inverse CDF is expensive or unavailable

**Example - Exponential distribution:**
- CDF: $F(x) = 1 - e^{-\lambda x}$ for $x \geq 0$
- Inverse: $F^{-1}(u) = -\frac{1}{\lambda}\log(1-u)$
- $\lambda$: Rate parameter of exponential distribution

In [ ]:
def sample_by_inversion(n, inv_cdf_fn, rng=None):
    """
    Generate samples using the inversion method.
    
    Parameters:
    -----------
    n : int
        Number of samples to generate
    inv_cdf_fn : callable
        Inverse cumulative distribution function F^{-1}(u)
        Input: u in [0, 1]
        Output: x from target distribution
    rng : numpy.random.Generator, optional
        Random number generator. If None, creates a new default generator.
    
    Returns:
    --------
    samples : numpy.ndarray, shape (n,)
        Samples from the target distribution
    """
    rng = np.random.default_rng() if rng is None else rng
    u = rng.uniform(0, 1, size=n)  # Generate uniform samples U ~ Uniform(0,1)
    return inv_cdf_fn(u)            # Apply inverse CDF: X = F^{-1}(U)

In [ ]:
# Example: Sample from Exponential(lambda=2) using inversion
rng = make_rng(seed=42)
lambda_param = 2.0

# Inverse CDF for Exponential distribution: F^{-1}(u) = -log(1-u)/lambda
inv_cdf = lambda u: -np.log(1 - u) / lambda_param

samples = sample_by_inversion(n=1000, inv_cdf_fn=inv_cdf, rng=rng)
print(f"Mean (expected ~{1/lambda_param:.2f}):", np.mean(samples))
print("First 5 samples:", samples[:5])

### Rejection Sampling

**What is this?**
A method to sample from a complex target distribution $f(x)$ by using a simpler proposal distribution $g(x)$.

**Algorithm:**
1. Sample $x$ from proposal distribution $g$
2. Sample $u \sim \text{Uniform}(0,1)$
3. Accept $x$ if $u \leq \frac{f(x)}{M \cdot g(x)}$, otherwise reject and try again
4. Repeat until you have enough accepted samples

**Variables:**
- $x$: Candidate sample from proposal distribution $g$
- $f(x)$: Target probability density function (PDF) we want to sample from
- $g(x)$: Proposal PDF, easier to sample from
- $M$: Envelope constant such that $f(x) \leq M \cdot g(x)$ for all $x$
- $u$: Uniform random variable on $(0, 1)$ for accept/reject decision
- $\frac{f(x)}{M \cdot g(x)}$: Acceptance probability for candidate $x$

**Key Requirements:**
- Need constant $M$ such that $f(x) \leq M \cdot g(x)$ for all $x$ (envelope condition)
- Smaller $M$ = Higher acceptance rate = More efficient
- $g(x)$ should be easy to sample from and evaluate
- Must be able to evaluate $f(x)$ (up to a normalizing constant)

**When to use:**
- Target distribution $f$ is hard to sample from directly
- But you can evaluate its PDF
- Common when inverse CDF is not available

**Efficiency:**
- Acceptance rate = $1/M$
- If $M$ is too large, you'll reject most samples (wasteful!)
- Choose $g$ to closely match $f$ for better efficiency

In [ ]:
def rejection_sample(n, sample_g, pdf_f, pdf_g, M, rng=None, max_tries=10_000_000):
    """
    Generate samples from target distribution f using rejection sampling.
    
    Parameters:
    -----------
    n : int
        Number of samples to generate
    sample_g : callable
        Function to sample from proposal distribution g
        Input: rng (random number generator)
        Output: one sample x from g
    pdf_f : callable
        Target probability density function f(x)
        Input: x (sample point)
        Output: density value f(x) (can be unnormalized)
    pdf_g : callable
        Proposal probability density function g(x)
        Input: x (sample point)
        Output: density value g(x)
    M : float
        Envelope constant such that f(x) <= M * g(x) for all x
        Smaller M = higher acceptance rate
    rng : numpy.random.Generator, optional
        Random number generator. If None, creates a new default generator.
    max_tries : int, default=10_000_000
        Maximum number of attempts before raising error (prevents infinite loops)
    
    Returns:
    --------
    samples : numpy.ndarray, shape (n,)
        Accepted samples from the target distribution f
    
    Raises:
    -------
    RuntimeError
        If unable to generate n samples within max_tries attempts
    """
    rng = np.random.default_rng() if rng is None else rng
    out = []
    tries = 0
    while len(out) < n and tries < max_tries:
        x = sample_g(rng)                          # Sample candidate from proposal g
        u = rng.uniform(0, 1)                      # Sample uniform for accept/reject
        accept_prob = pdf_f(x) / (M * pdf_g(x))   # Compute acceptance probability
        if u <= accept_prob:                       # Accept with probability f(x)/(M*g(x))
            out.append(x)
        tries += 1
    if len(out) < n:
        raise RuntimeError("Rejection sampling failed: increase max_tries or fix M/proposal.")
    return np.array(out)

In [ ]:
# Example: Sample from Beta(2,5) using rejection sampling with Uniform proposal
from scipy import stats

rng = make_rng(seed=42)

# Target: Beta(2, 5) distribution on [0, 1]
pdf_f = lambda x: stats.beta.pdf(x, 2, 5)

# Proposal: Uniform(0, 1)
sample_g = lambda rng: rng.uniform(0, 1)
pdf_g = lambda x: 1.0  # Uniform(0,1) has constant density 1

# Find envelope constant M (max of Beta(2,5) density)
M = stats.beta.pdf(stats.beta.mode(2, 5), 2, 5)

samples = rejection_sample(n=1000, sample_g=sample_g, pdf_f=pdf_f, pdf_g=pdf_g, M=M, rng=rng)
print(f"Mean (expected ~{2/(2+5):.3f}):", np.mean(samples))
print("First 5 samples:", samples[:5])

### Importance Sampling

**What is this?**
A variance reduction technique for Monte Carlo estimation by sampling from a better-chosen distribution.

**Problem:** Standard MC can be inefficient when the important region (where $h(x)f(x)$ is large) is rare.

**Solution:** Sample from proposal $g(x)$ that concentrates on important regions.

**Formula:** 
$$E_f[h(X)] = \int h(x)f(x)dx = \int h(x)\frac{f(x)}{g(x)}g(x)dx = E_g\left[h(X)\frac{f(X)}{g(X)}\right]$$

**Variables:**
- $X$: Random variable
- $f(x)$: Target probability density function
- $g(x)$: Proposal/importance density function (we sample from this)
- $h(x)$: Function we want to compute expectation of
- $w(x) = \frac{f(x)}{g(x)}$: Importance weight for sample $x$
- $E_f[\cdot]$: Expectation with respect to distribution $f$
- $E_g[\cdot]$: Expectation with respect to distribution $g$

**Algorithm:**
1. Sample $x_1, \ldots, x_n$ from proposal $g$
2. Compute importance weights: $w_i = \frac{f(x_i)}{g(x_i)}$ for $i=1,\ldots,n$
3. Estimate: $\hat{E}[h(X)] = \frac{1}{n}\sum_{i=1}^{n} h(x_i)w_i$

**When to use:**
- Rare event simulation (e.g., estimating tail probabilities)
- When you can't sample directly from $f$ but can evaluate it
- To reduce variance compared to standard MC
- When $g$ can be chosen to match $|h(x)|f(x)$

**Key insight:** Good proposal $g$ puts more samples where $|h(x)f(x)|$ is large, reducing variance

In [ ]:
def importance_sampling(n, sample_g, pdf_f, pdf_g, h_fn, rng=None):
    """
    Estimate E_f[h(X)] using importance sampling with proposal g.
    
    Parameters:
    -----------
    n : int
        Number of samples to draw from proposal distribution
    sample_g : callable
        Function to sample from proposal distribution g
        Input: rng (random number generator)
        Output: one sample x from g
    pdf_f : callable
        Target probability density function f(x)
        Input: x (sample point)
        Output: density value f(x)
    pdf_g : callable
        Proposal probability density function g(x)
        Input: x (sample point)
        Output: density value g(x)
    h_fn : callable
        Function to compute expectation of
        Input: x (sample point)
        Output: h(x) value
    rng : numpy.random.Generator, optional
        Random number generator. If None, creates a new default generator.
    
    Returns:
    --------
    estimate : float
        Importance sampling estimate of E_f[h(X)]
        Computed as: (1/n) * sum(h(x_i) * w_i) where w_i = f(x_i)/g(x_i)
    weights : numpy.ndarray, shape (n,)
        Importance weights w_i = f(x_i)/g(x_i) for each sample
    samples : numpy.ndarray, shape (n,)
        Generated samples x_1, ..., x_n from proposal distribution g
    """
    rng = np.random.default_rng() if rng is None else rng
    
    # Step 1: Generate samples from proposal g
    samples = [sample_g(rng) for _ in range(n)]
    
    # Step 2: Compute importance weights w_i = f(x_i) / g(x_i)
    weights = np.array([pdf_f(x) / pdf_g(x) for x in samples])
    
    # Step 3: Evaluate function h at each sample
    h_values = np.array([h_fn(x) for x in samples])
    
    # Step 4: Compute weighted average
    estimate = float(np.mean(weights * h_values))
    
    return estimate, weights, np.array(samples)

In [ ]:
# Example: Estimate E[X^2] for X ~ N(0,1) using importance sampling with N(2,1) proposal
from scipy import stats

rng = make_rng(seed=42)

# Target: N(0, 1)
pdf_f = lambda x: stats.norm.pdf(x, 0, 1)

# Proposal: N(2, 1)  
sample_g = lambda rng: rng.normal(2, 1)
pdf_g = lambda x: stats.norm.pdf(x, 2, 1)

# Function: h(x) = x^2
h_fn = lambda x: x**2

estimate, weights, samples = importance_sampling(n=5000, sample_g=sample_g, pdf_f=pdf_f, pdf_g=pdf_g, h_fn=h_fn, rng=rng)
print(f"Estimate of E[X^2] (expected = 1.0): {estimate:.4f}")
print(f"Mean weight: {np.mean(weights):.4f}")

### Monte-Carlo Integration

**What is this?**
A method to estimate expectations (integrals) using random sampling instead of analytical computation.

**Goal:** Estimate $E[h(X)] = \int h(x)f(x)dx$

**Variables:**
- $X$: Random variable with distribution $f$
- $h(x)$: Function we want to compute expectation of
- $f(x)$: Probability density function of $X$
- $x_1, x_2, \ldots, x_n$: Random samples drawn from $f$
- $\bar{h} = \frac{1}{n}\sum_{i=1}^{n} h(x_i)$: Sample average (MC estimator)
- $\mu = E[h(X)]$: True expectation (what we're estimating)
- $n$: Number of samples

**Algorithm:**
1. Draw samples $x_1, x_2, \ldots, x_n$ from distribution $f$
2. Compute $h(x_i)$ for each sample
3. Average: $\hat{E}[h(X)] = \frac{1}{n}\sum_{i=1}^{n} h(x_i)$

**Why it works:**
- Law of Large Numbers: Sample average $\bar{h}$ converges to true expectation $\mu$ as $n \to \infty$
- Central Limit Theorem: Error decreases as $O(1/\sqrt{n})$
- Standard error: $SE = \frac{\sigma}{\sqrt{n}}$ where $\sigma^2 = \text{Var}(h(X))$

**When to use:**
- High-dimensional integrals (curse of dimensionality: grid methods scale as $O(m^d)$, MC scales as $O(n)$)
- Complex functions where analytical integration is impossible
- Estimating probabilities, expected values, risk measures, etc.

**Example:** Estimating $\pi$ by sampling points uniformly in $[-1,1]^2$ and counting how many fall inside unit circle

In [ ]:
def monte_carlo_expectation(samples, h_fn):
    """
    Estimate the expectation E[h(X)] using Monte Carlo integration.
    
    Parameters:
    -----------
    samples : array-like, shape (n,)
        Random samples x_1, ..., x_n drawn from target distribution f
    h_fn : callable
        Function to compute expectation of
        Input: x (single sample)
        Output: h(x) value
    
    Returns:
    --------
    estimate : float
        Monte Carlo estimate of E[h(X)]
        Computed as: (1/n) * sum(h(x_i))
    vals : numpy.ndarray, shape (n,)
        Array of h(x_i) values for each sample
        Useful for computing variance, confidence intervals, etc.
    """
    samples = np.asarray(samples)
    vals = np.array([h_fn(x) for x in samples])  # Evaluate h at each sample
    return float(np.mean(vals)), np.asarray(vals, float)  # Return mean and all values

In [ ]:
# Example: Estimate E[X^2] for X ~ N(0,1) using Monte Carlo
rng = make_rng(seed=42)
samples = rng.normal(0, 1, size=10000)

# Function: h(x) = x^2
h_fn = lambda x: x**2

estimate, h_values = monte_carlo_expectation(samples, h_fn)
print(f"MC estimate of E[X^2] (expected = 1.0): {estimate:.4f}")
print(f"Standard error: {np.std(h_values)/np.sqrt(len(h_values)):.4f}")

### Hoeffding's Inequality

**What is this?**
A probability inequality that gives confidence bounds for the mean of bounded random variables.

**Formula:** For $n$ i.i.d. samples $X_i \in [a,b]$ with sample mean $\bar{X}$:
$$P(|\bar{X} - \mu| \geq \epsilon) \leq 2e^{-2n\epsilon^2/(b-a)^2}$$

**Variables:**
- $X_i$: Independent and identically distributed random variables, each in $[a, b]$
- $\bar{X} = \frac{1}{n}\sum_{i=1}^{n} X_i$: Sample mean
- $\mu = E[X_i]$: True population mean
- $n$: Number of samples
- $\epsilon$: Tolerance/error margin (how far sample mean can be from true mean)
- $[a, b]$: Known bounds on the random variables ($a \leq X_i \leq b$)
- $\alpha$: Significance level (probability of error)
- $1 - \alpha$: Confidence level (e.g., 0.95 for 95% confidence)

**Confidence Interval:**
With probability $1-\alpha$, the true mean $\mu$ is in:
$$\bar{X} \pm (b-a)\sqrt{\frac{\log(2/\alpha)}{2n}}$$

Equivalently: $\mu \in \left[\bar{X} - \delta, \bar{X} + \delta\right]$ where $\delta = (b-a)\sqrt{\frac{\log(2/\alpha)}{2n}}$

**Key Properties:**
- Works for ANY bounded distribution (no normality or other assumptions!)
- Confidence interval width: $2\delta = 2(b-a)\sqrt{\frac{\log(2/\alpha)}{2n}}$
- Width decreases as $O(1/\sqrt{n})$ (doubling samples reduces width by $\sqrt{2}$)
- Wider range $(b-a)$ = Wider confidence interval
- Smaller $\alpha$ (higher confidence) = Wider interval

**When to use:**
- When you don't know the distribution (distribution-free)
- When you need guaranteed bounds (not asymptotic)
- When variables are naturally bounded
- Trade-off: Conservative (wider than parametric CIs like t-intervals)

In [ ]:
import math

def hoeffding_ci(values, a, b, alpha=0.05):
    """
    Compute confidence interval for the mean using Hoeffding's inequality.
    
    Parameters:
    -----------
    values : array-like, shape (n,)
        Sample observations, each value should be in [a, b]
    a : float
        Lower bound on the random variables (minimum possible value)
    b : float
        Upper bound on the random variables (maximum possible value)
    alpha : float, default=0.05
        Significance level (1 - confidence level)
        E.g., alpha=0.05 gives 95% confidence interval
    
    Returns:
    --------
    confidence_interval : tuple (float, float)
        (lower_bound, upper_bound) of the confidence interval
        With probability 1-alpha, true mean is in this interval
    mean : float
        Sample mean (point estimate of true mean)
    radius : float
        Half-width of confidence interval (margin of error)
        Formula: delta = (b-a) * sqrt(log(2/alpha) / (2n))
    """
    values = np.asarray(values, float)
    n = len(values)                                          # Number of samples
    mean = float(np.mean(values))                           # Sample mean
    radius = (b - a) * math.sqrt(math.log(2/alpha) / (2*n)) # CI half-width
    return (mean - radius, mean + radius), mean, radius

In [ ]:
# Example: Compute 95% confidence interval for mean of bounded random variables
rng = make_rng(seed=42)
# Generate samples from Uniform(0, 10)
values = rng.uniform(0, 10, size=100)

ci, mean, radius = hoeffding_ci(values, a=0, b=10, alpha=0.05)
print(f"Sample mean: {mean:.3f}")
print(f"95% Confidence interval: [{ci[0]:.3f}, {ci[1]:.3f}]")
print(f"CI width: {2*radius:.3f}")
print(f"True mean (for Uniform(0,10)): 5.0")